In [1]:
import nltk
import pandas
import pickle
from re import sub
from os import path, mkdir
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to C:\Users\SAM-
[nltk_data]     Tech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SAM-
[nltk_data]     Tech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SAM-
[nltk_data]     Tech\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\SAM-
[nltk_data]     Tech\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
!wget http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
!tar -xf MovieSummaries.tar.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'MovieSummaries.tar.gz'


In [4]:
df_english = pandas.read_csv("./MovieSummarise/plot_summaries.txt",
                             delimiter="\t",
                             names=["id", "text"])
df_english.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42302 entries, 0 to 42301
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      42302 non-null  object
 1   text    42302 non-null  object
dtypes: object(2)
memory usage: 661.1+ KB


In [5]:
df_english["text"]

0        Shlykov, a hard-working taxi driver and Lyosha...
1        The nation of Panem consists of a wealthy Capi...
2        Poovalli Induchoodan  is sentenced for six yea...
3        The Lemon Drop Kid , a New York City swindler,...
4        Seventh-day Adventist Church pastor Michael Ch...
                               ...                        
42297    The story is about Reema , a young Muslim scho...
42298    In 1928 Hollywood, director Leo Andreyev  look...
42299    American Luthier focuses on Randy Parsons’ tra...
42300    Abdur Rehman Khan , a middle-aged dry fruit se...
42301    1940 - Operation Dynamo has just taken place. ...
Name: text, Length: 42302, dtype: object

In [6]:
sentences =df_english["text"][:]
sentences = [sub('http://\S+|https://\S+', '', sentence) for sentence in sentences]
sentences= [sub(r'[^\w\s]', '', sentence) for sentence in sentences ]

# NLTK

In [7]:
#tokenization
en_tokenized = [word_tokenize(sentence) for sentence in sentences]

In [8]:
#removing stop words
stop_words = set(stopwords.words('english'))
filtered_sentence=[]*len(en_tokenized)
for i in range(len(en_tokenized)):
  filtered_sentence.append([])
  for word in en_tokenized[i]:
    if word.lower() not in stop_words:
      filtered_sentence[i].append(word)


In [9]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
for i in range(len(filtered_sentence)):
    for j in range(len(filtered_sentence[i])):
        filtered_sentence[i][j] = stemmer.stem(filtered_sentence[i][j])
        filtered_sentence[i][j] = lemmatizer.lemmatize(filtered_sentence[i][j])


In [10]:
flat = []
for sentence in filtered_sentence:
  flat.extend(sentence)

# indexing

In [11]:
def indexer(start,end,name):
    posting=dict()
    c=0
    for i, sentence in enumerate(filtered_sentence[start:end]):
        for token in set(sentence):
            num = sentence.count(token)
            posting.setdefault(token, []).append((i,num))
            c+=1
    for key,value in posting.items():
        posting[key]= sorted(value,key=lambda x:x[1],reverse =True)
    f=open(f"{name}.pickle","wb")
    pickle.dump(obj = posting,file = f)
    f.close()
    return posting


In [12]:
length = len(filtered_sentence)
posting: dict[str, list[tuple[int, int]]] = indexer(0,length,"en_indexes")

# Information Retrieval

In [13]:
words = sorted(list(set(flat)))

In [14]:
sentences_count = len(filtered_sentence)
words_count=len(words)
PARTS= 70
UNIT_SIZE= words_count / PARTS


In [15]:

#divided by 20:
if not path.exists("./booleans/"):
    mkdir("./booleans/")
for n in range(PARTS):
    matrix: list[list[int]] = []
    start = int(n * (words_count / PARTS))
    end = min(int((n+1) * (words_count / PARTS)),words_count)
    print(f"{n}. {words[start]} -> {words[end-1]}")
    for i, word in enumerate(words[start:end]):
        matrix.append([False]*sentences_count)
        for index,count in posting[word]:
            matrix[i][index]=True
    with open(f"./booleans/boolean{n}.pickle","bw") as file:
        pickle.dump(obj=matrix,file=file)


0. 0 -> 2600
1. 26000 -> abow
2. abra -> alaya
3. alazra -> animalrel
4. animalright -> asad
5. asada -> bagladi
6. bagley -> beedi
7. beedinow -> birchal
8. birckmay -> bowler
9. bowlerhat -> busboyz
10. busboyzmdashstev -> castwikit
11. castworkallmoviecomaccessdateimdbtbq061110 -> chico
12. chico39 -> coaticook
13. coatjohn -> costumeoutfit
14. costumeref -> dairi
15. dairyfarm -> denahi
16. denali -> distaff
17. distanc -> duisburg
18. dujanara -> emmett
19. emmi -> expector
20. expedi -> fever
21. feverdream -> fortyf
22. fortyfour -> gangstaz
23. gangster -> glumli
24. glumlook -> grunbaum
25. grunberg -> hant
26. hanthawadi -> heureux
27. heurtebis -> housekey
28. houselat -> incalcublii
29. incalcul -> izuchi
30. izuhura -> jonestyp
31. jong -> kashipur
32. kashiram -> kleiner
33. kleinercast -> kwast
34. kway -> lek
35. lekamg -> longrang
36. longrehears -> magpaliwanag
37. magpi -> marotta
38. marou -> meinhof
39. meinik -> misandri
40. misandrist -> moustakia
41. moustapha -

boolean

In [16]:
def get_binary(s:str) -> list[int]:
    filtered_word = stemmer.stem(s.lower())
    filtered_word = lemmatizer.lemmatize(filtered_word)
    index = words.index(filtered_word)+1
    file_number= int(index/UNIT_SIZE)
    index= int(index%UNIT_SIZE)    
    with open(f"./booleans/boolean{file_number}.pickle","br") as file:
        matrix= pickle.load(file=file)
    return matrix[index]

In [17]:
def AandB(a:list[int] , b:list[int]) -> list[int]:
    return [a[i] and b[i] for i in range(len(a))]


In [18]:
def AorB(a:list[int],b:list[int])->list[int]:
    return [a[i] or b[i] for i in range(len(a))]


In [19]:
def notA(a:list[int])->list[int]:
    return [not i for i in a]

In [20]:
def show(a:list[int])->None:
    for i in range(len(a)):
        if a[i]:
            print("id = ",df_english["id"][i], df_english["text"][i])

In [21]:
def parseString(string:str)-> list[str]:
    tokens=[]
    token = ""
    for char in string.strip():
        if char in '(^|!) ':
            if len(token):
                tokens.append(token)
                token=""
            if char!=" ":
                tokens.append(char)
        else:
            token+=char
    tokens.append(token)
    return tokens 

In [22]:
def convert_to_binary(tokens:list[str])-> list[str|list[int]]:
    res = []
    for token in tokens:
        if token in "!(^)|":
            res.append(token)
        else:
            print(f"Finding the vector of {token}.")
            res.append(get_binary(token))
    return res

In [23]:
def process(tokens:list)->list[int]:
    temp = tokens.copy()
    index =0
    while index<len(temp):
        if temp[index] =="(":
            close_par_index=index+1
            inner_pars=0
            while close_par_index<len(temp):
                if temp[close_par_index]=="(":
                    inner_pars += 1
                elif temp[close_par_index]==")":
                    if inner_pars==0:
                        break
                    inner_pars -= 1
                close_par_index +=1
            answer = process(temp[index+1:close_par_index])
            del temp[index:close_par_index + 1]
            temp.insert(index,answer)
        index+=1
    index=0
    while index<len(temp):
        if temp[index] == "!":
            answer = notA(temp[index + 1])
            del temp[index:index + 2]
            temp.insert(index, answer)
        index+=1
    index =0
    while index<len(temp):
        if temp[index] == "^":
            answer = AandB(temp[index - 1], temp[index + 1])
            del temp[index - 1:index + 2]
            temp.insert(index - 1, answer)
            index -=1
        elif temp[index] == "|":
            answer = AorB(temp[index - 1], temp[index + 1])
            del temp[index - 1:index + 2]
            temp.insert(index - 1, answer)
            index -= 1
        index+=1
    return temp[0]


In [24]:
def find():
    query = input("enter your search string: ")
    if query:
        print("Parsing your input...")
        tokens = parseString(query)
        print("Convert words to its vectors...")
        binaries = convert_to_binary(tokens)
        print("Searching for the results...")
        answer = process(binaries)
        print("Your descriptions are here:")
        show(answer)

In [25]:
''' 
    use ^ for and, | for or, ! for not and paranthesis for better result.
    e.g. (first ^ black ^ jigsaw) | (conquerors ^ !buddha)
'''
find()

Parsing your input...
Convert words to its vectors...
Finding the vector of first.
Finding the vector of black.
Finding the vector of jigsaw.
Finding the vector of conquerors.
Finding the vector of buddha.
Searching for the results...
Your descriptions are here:
id =  8471210 In 1942, a 10 year old boy named Timmy plays with a jigsaw puzzle of a nude woman when his mother walks in. She chastises him for it. The mother orders Timmy to get a plastic bag because "I'm gonna burn everything." Timmy gruesomely murders her with an axe instead of getting the plastic bag. The police later arrive at the house and Timmy is finishing the puzzle. The police break into the house through the front door. Timmy then hides in one of the closets. The police then enter into the bedroom, they find a bloody mess all over the room and also find the mother's head inside one of the closets. The police hear Timmy weeping in the another closet, so they open the door and find him covered in blood. The police ques